# Objetivo

En este trabajo vamos a tratar de implementar el modelo de evaluación de riesgo de la epidemia desarrollado por el equipo de la [Computational Biology and Complex Systems; Universitat Politècnica de Catalunya - BarcelonaTech](https://biocomsc.upc.edu/en/shared/20200412_report_web_27.pdf) . Para obtener los datos epidemiológicos nos apoyamos en el grupo de trabajo [#escovid19data](https://github.com/montera34/escovid19data).


@author: Manuel H. Arias 

@Twitter: @walyt

@mail: mharias@me.com



## Importación de librerias estándares para DAE

In [2]:
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from matplotlib import cm
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from matplotlib.dates import (YEARLY, MONTHLY, DateFormatter, MonthLocator,DayLocator,
                              rrulewrapper, RRuleLocator, drange)
import matplotlib.image as mpimg
from matplotlib.animation import FuncAnimation
from matplotlib.offsetbox import TextArea, DrawingArea, OffsetImage, AnnotationBbox
import matplotlib.colors as colors

import numpy as np
from datetime import datetime,timedelta
import seaborn as sns
%matplotlib inline

import urllib.request

import matplotlib.image as mpimg
from matplotlib.animation import FuncAnimation
from matplotlib.offsetbox import TextArea, DrawingArea, OffsetImage, AnnotationBbox

## Descargamos la información

Hacemos en primer lugar una actualización de parámetros y preparación de variables que necesitaremos durante el ejercicio

In [3]:
pd.options.display.max_rows = 999 #Variable de contexto para permitir la presentación de datos por pantalla
pd.set_option('display.max_columns', None)

In [4]:
#url de este notebook en github
github_url = 'https://github.com/mharias/covid/blob/master/graficos_escovid19data.ipynb'

In [5]:
hashtag_fuente='#escovid19data' #hashtag de la fuente para los créditos

In [6]:
#url de la fuente de datos
path_montera34='https://github.com/montera34/escovid19data/blob/master/data/output/covid19-provincias-spain_consolidated.csv?raw=true'

Leemos los datos en un `pandas`

In [7]:
df = pd.read_csv(path_montera34)

veamos una rápida descripción de la información:

In [8]:
df.describe()

,ine_code,new_cases,PCR,TestAc,activos,hospitalized,intensive_care,deceased,cases_accumulated,cases_accumulated_PCR,recovered,num_casos,num_casos_prueba_pcr,num_casos_prueba_test_ac,num_casos_prueba_otras,num_casos_prueba_desconocida,poblacion,cases_per_cienmil,intensive_care_per_1000000,deceassed_per_100000,hospitalized_per_100000,cases_14days,cases_7days,cases_PCR_14days,cases_PCR_7days,daily_cases,daily_cases_avg7,daily_cases_PCR,daily_cases_PCR_avg7,daily_deaths,daily_deaths_inc,daily_deaths_avg3,daily_deaths_avg7,deaths_last_week,num_casos_prueba_pcr_avg7
count,13478.000000,4219.000000,3781.000000,940.000000,1524.000000,9581.000000,9444.000000,10729.000000,6917.000000,8087.000000,6940.000000,13468.000000,13468.00000,13468.000000,13468.000000,13468.000000,1.347800e+04,6917.000000,9444.000000,10729.000000,9581.000000,6086.000000,6468.000000,7373.000000,7699.000000,6757.000000,6071.000000,8631.000000,8267.000000,10664.000000,9990.0,10004.000000,10365.000000,8997.000000,13156.000000
mean,26.505416,55.757525,100.830733,6211.639362,1090.250000,178.971297,24.456057,578.239911,5390.596068,8342.986398,1722.850844,69.097268,64.73515,1.535417,1.434140,1.392560,9.038635e+05,485.558495,2.625023,52.647850,20.187207,1165.483569,574.597866,1383.419232,692.933368,80.449904,82.266595,96.015410,94.666614,3.849306,inf,3.937545,3.821081,29.132155,65.759973
std,15.010464,128.478359,195.230650,19250.346611,1565.903329,788.975518,90.430526,1784.224161,13557.108329,23340.679885,2428.416165,257.030656,249.47040,6.592804,28.959428,9.839137,1.176461e+06,523.761794,3.888182,50.792749,42.415130,2589.998180,1311.940836,4244.342581,2126.586716,201.733846,191.663414,322.854011,294.033761,20.476369,NaN,20.751678,20.189236,150.864255,240.257836
min,1.000000,-67.000000,-67.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,8.477700e+04,0.000000,0.000000,0.000000,0.000000,-54.000000,-82.000000,-62.000000,-99.000000,-106.000000,-11.700000,-84.000000,-12.700000,-22.000000,-30.8,-1.300000,-3.300000,-4.000000,0.000000
25%,14.000000,3.000000,6.000000,3.000000,167.500000,11.000000,2.000000,90.000000,702.000000,1451.500000,416.000000,1.000000,1.00000,0.000000,0.000000,0.000000,3.167980e+05,125.970000,0.297500,12.740000,2.300000,122.000000,52.750000,44.000000,21.000000,5.000000,7.300000,2.000000,3.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.400000
50%,27.000000,20.000000,35.000000,22.000000,510.000000,42.000000,6.000000,203.000000,2016.000000,2949.000000,1165.500000,12.000000,9.00000,0.000000,0.000000,0.000000,5.810780e+05,314.910000,1.250000,33.670000,8.210000,496.000000,232.500000,260.000000,124.000000,27.000000,32.900000,15.000000,17.300000,0.000000,0.0,0.300000,0.500000,4.000000,9.900000
75%,40.000000,59.000000,106.000000,442.000000,1260.250000,123.000000,19.000000,370.000000,4410.000000,5502.500000,2101.250000,58.000000,52.00000,0.000000,0.000000,0.000000,1.022800e+06,678.040000,3.380000,82.100000,22.470000,1097.000000,542.000000,1083.000000,555.000000,76.000000,77.100000,74.000000,76.000000,2.000000,0.9,2.000000,2.000000,16.000000,54.625000
max,52.000000,1992.000000,2149.000000,115043.000000,10275.000000,15227.000000,1528.000000,11778.000000,137948.000000,283753.000000,27721.000000,6743.000000,6729.00000,128.000000,1306.000000,261.000000,6.663394e+06,3274.780000,28.720000,244.070000,474.750000,29954.000000,17440.000000,52397.000000,28142.000000,3374.000000,2491.400000,6787.000000,4020.300000,377.000000,inf,350.700000,339.000000,2373.000000,4164.100000


y un muestreo de valores y de algunas columnas de interés:

In [9]:
df.head()

,date,province,ine_code,ccaa,new_cases,PCR,TestAc,activos,hospitalized,intensive_care,deceased,cases_accumulated,cases_accumulated_PCR,recovered,num_casos,num_casos_prueba_pcr,num_casos_prueba_test_ac,num_casos_prueba_otras,num_casos_prueba_desconocida,poblacion,cases_per_cienmil,intensive_care_per_1000000,deceassed_per_100000,hospitalized_per_100000,cases_14days,cases_7days,cases_PCR_14days,cases_PCR_7days,daily_cases,daily_cases_avg7,daily_cases_PCR,daily_cases_PCR_avg7,daily_deaths,daily_deaths_inc,daily_deaths_avg3,daily_deaths_avg7,deaths_last_week,num_casos_prueba_pcr_avg7,source_name,source,comments
0,2020-01-31,Albacete,2,Castilla - La Mancha,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,388167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ISCIII RENAVE,https://cnecovid.isciii.es/covid19/resources/d...,NaN
1,2020-01-31,Alicante/Alacant,3,Comunitat Valenciana,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1858683,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ISCIII RENAVE,https://cnecovid.isciii.es/covid19/resources/d...,NaN
2,2020-01-31,Almería,4,Andalucía,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,716820,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ISCIII RENAVE,https://cnecovid.isciii.es/covid19/resources/d...,NaN
3,2020-01-31,Araba/Álava,1,País Vasco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,331549,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ISCIII RENAVE,https://cnecovid.isciii.es/covid19/resources/d...,NaN
4,2020-01-31,Asturias,33,"Asturias, Principado de",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1022800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ISCIII RENAVE,https://cnecovid.isciii.es/covid19/resources/d...,NaN


In [10]:
df.tail()

,date,province,ine_code,ccaa,new_cases,PCR,TestAc,activos,hospitalized,intensive_care,deceased,cases_accumulated,cases_accumulated_PCR,recovered,num_casos,num_casos_prueba_pcr,num_casos_prueba_test_ac,num_casos_prueba_otras,num_casos_prueba_desconocida,poblacion,cases_per_cienmil,intensive_care_per_1000000,deceassed_per_100000,hospitalized_per_100000,cases_14days,cases_7days,cases_PCR_14days,cases_PCR_7days,daily_cases,daily_cases_avg7,daily_cases_PCR,daily_cases_PCR_avg7,daily_deaths,daily_deaths_inc,daily_deaths_avg3,daily_deaths_avg7,deaths_last_week,num_casos_prueba_pcr_avg7,source_name,source,comments
13473,2020-10-16,Salamanca,37,Castilla y León,96.0,NaN,NaN,NaN,117.0,23.0,453.0,NaN,10518.0,1833.0,NaN,NaN,NaN,NaN,NaN,330119,NaN,6.97,137.22,35.44,NaN,NaN,1557.0,852.0,NaN,NaN,96.0,121.7,2.0,0.4,2.3,2.1,15.0,NaN,Junta de Castilla y León,https://analisis.datosabiertos.jcyl.es/explore...,NaN
13474,2020-10-16,Segovia,40,Castilla y León,76.0,NaN,NaN,NaN,34.0,3.0,228.0,NaN,6021.0,1040.0,NaN,NaN,NaN,NaN,NaN,153129,NaN,1.96,148.89,22.20,NaN,NaN,559.0,315.0,NaN,NaN,76.0,45.0,0.0,0.0,1.0,0.6,4.0,NaN,Junta de Castilla y León,https://analisis.datosabiertos.jcyl.es/explore...,NaN
13475,2020-10-16,Soria,42,Castilla y León,16.0,NaN,NaN,NaN,20.0,4.0,132.0,NaN,3838.0,532.0,NaN,NaN,NaN,NaN,NaN,88636,NaN,4.51,148.92,22.56,NaN,NaN,171.0,102.0,NaN,NaN,16.0,14.6,0.0,0.0,0.0,0.3,2.0,NaN,Junta de Castilla y León,https://analisis.datosabiertos.jcyl.es/explore...,NaN
13476,2020-10-16,Valladolid,47,Castilla y León,239.0,NaN,NaN,NaN,198.0,44.0,507.0,NaN,15230.0,2725.0,NaN,NaN,NaN,NaN,NaN,519546,NaN,8.47,97.59,38.11,NaN,NaN,2493.0,1197.0,NaN,NaN,239.0,171.0,7.0,1.4,3.3,3.3,23.0,NaN,Junta de Castilla y León,https://analisis.datosabiertos.jcyl.es/explore...,NaN
13477,2020-10-16,Zamora,49,Castilla y León,86.0,NaN,NaN,NaN,65.0,4.0,146.0,NaN,3526.0,604.0,NaN,NaN,NaN,NaN,NaN,172539,NaN,2.32,84.62,37.67,NaN,NaN,535.0,314.0,NaN,NaN,86.0,44.9,2.0,1.4,1.0,1.0,7.0,NaN,Junta de Castilla y León,https://analisis.datosabiertos.jcyl.es/explore...,NaN


In [11]:
df['province'].unique()

array(['Albacete', 'Alicante/Alacant', 'Almería', 'Araba/Álava',
       'Asturias', 'Ávila', 'Badajoz', 'Balears, Illes', 'Barcelona',
       'Bizkaia', 'Burgos', 'Cáceres', 'Cádiz', 'Cantabria',
       'Castellón/Castelló', 'Ceuta', 'Ciudad Real', 'Córdoba',
       'Coruña, A', 'Cuenca', 'Gipuzkoa', 'Girona', 'Granada',
       'Guadalajara', 'Huelva', 'Huesca', 'Jaén', 'León', 'Lleida',
       'Lugo', 'Madrid', 'Málaga', 'Melilla', 'Murcia', 'Navarra',
       'Ourense', 'Palencia', 'Palmas, Las', 'Pontevedra', 'Rioja, La',
       'Salamanca', 'Santa Cruz de Tenerife', 'Segovia', 'Sevilla',
       'Soria', 'Tarragona', 'Teruel', 'Toledo', 'Valencia/València',
       'Valladolid', 'Zamora', 'Zaragoza'], dtype=object)

Vamos a formatear correctamente la columna `Date`, para posteriormente ordenar el $pandas$ por fecha creciente:

In [12]:
df['date']=pd.to_datetime(df['date'])

In [13]:
df.columns

Index(['date', 'province', 'ine_code', 'ccaa', 'new_cases', 'PCR', 'TestAc',
       'activos', 'hospitalized', 'intensive_care', 'deceased',
       'cases_accumulated', 'cases_accumulated_PCR', 'recovered', 'num_casos',
       'num_casos_prueba_pcr', 'num_casos_prueba_test_ac',
       'num_casos_prueba_otras', 'num_casos_prueba_desconocida', 'poblacion',
       'cases_per_cienmil', 'intensive_care_per_1000000',
       'deceassed_per_100000', 'hospitalized_per_100000', 'cases_14days',
       'cases_7days', 'cases_PCR_14days', 'cases_PCR_7days', 'daily_cases',
       'daily_cases_avg7', 'daily_cases_PCR', 'daily_cases_PCR_avg7',
       'daily_deaths', 'daily_deaths_inc', 'daily_deaths_avg3',
       'daily_deaths_avg7', 'deaths_last_week', 'num_casos_prueba_pcr_avg7',
       'source_name', 'source', 'comments'],
      dtype='object')

# Gráfico de BiocomSC

## preparamos el Pandas

In [14]:
# Gráfico de BiocomSC


provincias=['Madrid']
datos = df.fillna(0,axis=1)
datos = datos[(datos['date']>=datetime(year=2020,month=3,day=1)) & (datos['province'].isin(provincias))]

clave_avg='daily_cases_PCR_avg7'
clave_ratio_avg = 'ratio_daily_cases_PCR_avg7'
color_avg = 'darkred'
color_titulos = 'navy'
color_diarios = 'royalblue'
clave_ratio = 'tasa_ia_7_dias'
AI='incidencia_acumulada_14'
casos_PCR_14='cases_PCR_14days'
casos_nuevos = 'daily_cases_PCR'
clave_casos_popu = 'casos_per_population'
ventana_IA=14
titulo = 'Incidencia Acumulada de 14 días de PCR+ por 100.000 hab \n Ratio de la IA día(0)/día(-7)'
texto_y1='IA a 14 días'
texto_y2='Ratio a 7 días'

un_dia=timedelta(days=1)
dos_dias=timedelta(days=2)
cinco_dias=timedelta(days=5)
seis_dias=timedelta(days=6)
siete_dias=timedelta(days=7)
  
for provincia in provincias:
    
    datos.loc[datos['province']==provincia,AI]=\
    datos.loc[datos['province']==provincia,casos_PCR_14]\
    /datos.loc[datos['province']==provincia,'poblacion']*100000

    datos.loc[datos['province']==provincia,clave_ratio] = \
    datos.loc[datos['province']==provincia][AI].pct_change(periods=7).add(1)    
    
    datos.loc[datos['province']==provincia,clave_casos_popu]=\
    datos.loc[datos['province']==provincia,casos_nuevos]\
    /datos.loc[datos['province']==provincia,'poblacion']*1000000

#datos = datos.set_index('date')[[casos_PCR_14,clave_ratio]]   
  
def calculo_p7(x):
    fecha_actual=x['date']
    #print (fecha_actual)
    if fecha_actual>datetime(year=2020,month=3,day=8):
        valor = (datos.loc[datos['date']==fecha_actual,'daily_cases_PCR'].values[0]+\
        datos.loc[datos['date']==(fecha_actual-un_dia),'daily_cases_PCR'].values[0]+\
        datos.loc[datos['date']==(fecha_actual-dos_dias),'daily_cases_PCR'].values[0])/\
        (datos.loc[datos['date']==(fecha_actual-cinco_dias),'daily_cases_PCR'].values[0]+\
         datos.loc[datos['date']==(fecha_actual-seis_dias),'daily_cases_PCR'].values[0]+\
         datos.loc[datos['date']==(fecha_actual-siete_dias),'daily_cases_PCR'].values[0])
    else:
        valor = 0
    return valor


datos['p7']=datos.apply(calculo_p7,axis=1)
datos['p7_avg7'] = datos['p7'].rolling(window=7).mean()
datos = datos.reset_index()
datos = datos.drop(['index'],axis=1)

In [15]:
color_rojo = sns.color_palette("bright", 10)[3]
color_verde = sns.color_palette("muted", 10)[2]
color_amarillo = sns.color_palette("YlOrRd", 10)[1]
eje_X = ' Incidencia Acumulada a 14 días'
eje_Y = 'Parámetro \u03C1'+'7'

fuente_modelo = 'https://biocomsc.upc.edu/en/shared/20200412_report_web_27.pdf'


y0, y1 = (-0.21951443768357135, 4.609803191354998)
#y= 0, 2

x0, x1 = (-26.666590629339943, 817.5248229355791)
#x0, x1 = 0,300
rango = np.arange(x0,x1,1)
def formateamos_eje(ax):
    '''
    función que formatea el gráfico con las características que queramos
    '''
    ax.set_facecolor('.8')
    ax.tick_params(labelsize=8, length=0)
    ax.grid(True, axis='x', color='white')
    ax.set_axisbelow(True)
    [spine.set_visible(False) for spine in ax.spines.values()]

    

def init():
    '''
    función que inicia el eje en el que dibujamos las barras
    
    '''
    ax.clear()
    ax.set_xlim(0,x1)
    ax.set_ylim(0,y1)
    formateamos_eje(ax)
    #ax.set_ylim(.2, 6.8)



def update(i):
    '''
    Función invocada para ir regenerando el gráfico, 
    Va actualizando añadiendo días (i incrementandose).
    
    '''
    for elemento in lista_elementos:
        elemento.remove()
    lista_elementos[:] = []
    
    
    #print (i)
    #ax.set_xlim(0,x1)
    #ax.set_ylim(0,y1)
    #colores = sns.color_palette("YlOrBr",i)
    colores = sns.color_palette("Blues",i)
    ax.scatter(datos[AI][:i],datos['p7_avg7'][:i],color=colores[:i:],marker='o',
               s=5)
    #ax.xaxis.set_major_locator(DayLocator([1]))
    #majorFmt = DateFormatter('%-d-%b')
    #ax.xaxis.set_major_formatter(majorFmt)
    ax.grid(True,axis='both')
    ax.yaxis.set_tick_params(labelsize=0,labelcolor=color_avg,width=0)
    ax.set_ylabel(eje_Y,size=14,color='black')
    ax.set_xlabel(eje_X,size=14,color='black')
    #ax4.set_yticks([])
    ax.yaxis.set_tick_params(labelsize=16,labelcolor=color_avg,width=0)
        
    ax.yaxis.set_tick_params(labelsize=12)
    ax.xaxis.set_tick_params(labelsize=12)
    
    for indice in range(i+1):
        if indice % 15 == 0 or indice==7 or indice == (len(datos.index)-1):
            try:
                color_texto = colores[indice-1]
            except:
                color_texto = 'black'
            style = dict(size=8, color=color_texto,weight='bold')
            texto = ax.annotate(datos.iloc[indice]['date'].strftime('%B %-d'),
                                xy=(datos.iloc[indice][AI],datos.iloc[indice]['p7_avg7']),
                                xycoords='data',
                                xytext=(-10, 10), 
                                textcoords='offset points',
                                **style)
            lista_elementos.append(texto)
    
    #date_str = datos.index[i].strftime('%B %-d, %Y')
    ax.hlines(1,x0,x1,colors='black',linestyles='dotted')
    
    ax.fill_between(rango, 0, 30/rango,
                 facecolor="blue", # The fill color
                 color=color_verde,       # The outline color
                 alpha=0.02) 
    ax.fill_between(rango, 30/rango, 100/rango,
                 facecolor='blue', # The fill color
                 color=color_amarillo,       # The outline color
                 alpha=0.02) 
    ax.fill_between(rango,100/rango,10,
                 facecolor="blue", # The fill color
                 color=color_rojo,       # The outline color
                 alpha=0.02)
    
    
    ax.set_title(f' Evolución de Empiric Potential Growth en Madrid\n Fuente del modelo @biocomsc \n Fuente de los datos #escovid19data', fontsize=12)


lista_elementos = []    
fig = plt.Figure(figsize=(10, 5), dpi=144)
ax = fig.add_subplot()


anim = FuncAnimation(fig=fig, func=update, init_func=init, frames=len(datos), 
                     interval=150, repeat=False)

#con interval puedes modular la velocidad del video, más alto --> mas lento

In [16]:
len(datos.index)

229

In [17]:
from IPython.display import HTML
html = anim.to_html5_video()
HTML(html)

In [18]:
anim.save('modelo_biocomsc_madrid.mp4',dpi=288)

# Gráfico de BiocomSC Badajoz

In [19]:
provincias=['Badajoz']
datos = df.fillna(0,axis=1)
datos = datos[(datos['date']>=datetime(year=2020,month=3,day=1)) & (datos['province'].isin(provincias))]

clave_avg='daily_cases_PCR_avg7'
clave_ratio_avg = 'ratio_daily_cases_PCR_avg7'
color_avg = 'darkred'
color_titulos = 'navy'
color_diarios = 'royalblue'
clave_ratio = 'tasa_ia_7_dias'
AI='incidencia_acumulada_14'
casos_PCR_14='cases_PCR_14days'
casos_nuevos = 'daily_cases_PCR'
casos_nuevos = 'new_cases'
clave_casos_popu = 'casos_per_population'
ventana_IA=14
titulo = 'Incidencia Acumulada de 14 días de PCR+ por 100.000 hab \n Ratio de la IA día(0)/día(-7)'
texto_y1='IA a 14 días'
texto_y2='Ratio a 7 días'

un_dia=timedelta(days=1)
dos_dias=timedelta(days=2)
cinco_dias=timedelta(days=5)
seis_dias=timedelta(days=6)
siete_dias=timedelta(days=7)
  
for provincia in provincias:
    
    datos.loc[datos['province']==provincia,AI]=\
    datos.loc[datos['province']==provincia,casos_nuevos].rolling(ventana_IA).sum()\
    /datos.loc[datos['province']==provincia,'poblacion']*100000

    datos.loc[datos['province']==provincia,clave_ratio] = \
    datos.loc[datos['province']==provincia][AI].pct_change(periods=7).add(1)    
    
    datos.loc[datos['province']==provincia,clave_casos_popu]=\
    datos.loc[datos['province']==provincia,casos_nuevos]\
    /datos.loc[datos['province']==provincia,'poblacion']*1000000

#datos = datos.set_index('date')[[casos_PCR_14,clave_ratio]]   
  
def calculo_p7(x):
    fecha_actual=x['date']
    #print (fecha_actual)
    if fecha_actual>datetime(year=2020,month=3,day=8):
        valor = (datos.loc[datos['date']==fecha_actual,casos_nuevos].values[0]+\
        datos.loc[datos['date']==(fecha_actual-un_dia),casos_nuevos].values[0]+\
        datos.loc[datos['date']==(fecha_actual-dos_dias),casos_nuevos].values[0])/\
        (datos.loc[datos['date']==(fecha_actual-cinco_dias),casos_nuevos].values[0]+\
         datos.loc[datos['date']==(fecha_actual-seis_dias),casos_nuevos].values[0]+\
         datos.loc[datos['date']==(fecha_actual-siete_dias),casos_nuevos].values[0])
    else:
        valor = 0
    return valor


datos['p7']=datos.apply(calculo_p7,axis=1)
datos['p7_avg7'] = datos['p7'].rolling(window=7).mean()
datos = datos.reset_index()
datos = datos.drop(['index'],axis=1)

<ipython-input-19-634a5fe5f01e>:49: RuntimeWarning: divide by zero encountered in double_scalars
  valor = (datos.loc[datos['date']==fecha_actual,casos_nuevos].values[0]+\
<ipython-input-19-634a5fe5f01e>:49: RuntimeWarning: invalid value encountered in double_scalars
  valor = (datos.loc[datos['date']==fecha_actual,casos_nuevos].values[0]+\


In [25]:
color_rojo = sns.color_palette("bright", 10)[3]
color_verde = sns.color_palette("muted", 10)[2]
color_amarillo = sns.color_palette("YlOrRd", 10)[1]
eje_X = ' Incidencia Acumulada a 14 días'
eje_Y = 'Parámetro \u03C1'+'7'

fuente_modelo = 'https://biocomsc.upc.edu/en/shared/20200412_report_web_27.pdf'


y0, y1 = (-0.21951443768357135, 4.609803191354998)
y= 0, 2

x0, x1 = (-26.666590629339943, 817.5248229355791)
x0, x1 = 0,300
rango = np.arange(x0,x1,1)
def formateamos_eje(ax):
    '''
    función que formatea el gráfico con las características que queramos
    '''
    ax.set_facecolor('.8')
    ax.tick_params(labelsize=8, length=0)
    ax.grid(True, axis='x', color='white')
    ax.set_axisbelow(True)
    [spine.set_visible(False) for spine in ax.spines.values()]

    

def init():
    '''
    función que inicia el eje en el que dibujamos las barras
    
    '''
    ax.clear()
    formateamos_eje(ax)
    #ax.set_ylim(.2, 6.8)



def update(i):
    '''
    Función invocada para ir regenerando el gráfico, 
    Va actualizando añadiendo días (i incrementandose).
    
    '''
    for elemento in lista_elementos:
        elemento.remove()
    lista_elementos[:] = []
    
    
    
    #print (i)
    ax.set_xlim(0,x1)
    ax.set_ylim(0,y1)
    #colores = sns.color_palette("YlOrBr",i)
    colores = sns.color_palette("Blues",i)
    ax.scatter(datos[AI][:i],datos['p7_avg7'][:i],color=colores[:i:],marker='o',
               s=5)
    #ax.xaxis.set_major_locator(DayLocator([1]))
    #majorFmt = DateFormatter('%-d-%b')
    #ax.xaxis.set_major_formatter(majorFmt)
    ax.grid(True,axis='both')
    ax.yaxis.set_tick_params(labelsize=0,labelcolor=color_avg,width=0)
    ax.set_ylabel(eje_Y,size=14,color='black')
    ax.set_xlabel(eje_X,size=14,color='black')
    #ax4.set_yticks([])
    ax.yaxis.set_tick_params(labelsize=16,labelcolor=color_avg,width=0)
        
    ax.yaxis.set_tick_params(labelsize=12)
    ax.xaxis.set_tick_params(labelsize=12)
    
    for indice in range(i+1):
        if indice % 15 == 0 or indice==7 or indice == (len(datos.index)-1):
            try:
                color_texto = colores[indice-1]
            except:
                color_texto = 'black'
            style = dict(size=8, color=color_texto,weight='bold')
            texto = ax.annotate(datos.iloc[indice]['date'].strftime('%B %-d'),
                                xy=(datos.iloc[indice][AI],datos.iloc[indice]['p7_avg7']),
                                xycoords='data',
                                xytext=(-10, 10), 
                                textcoords='offset points',
                                **style)
            lista_elementos.append(texto)
    
    
    #date_str = datos.index[i].strftime('%B %-d, %Y')
    ax.hlines(1,x0,x1,colors='black',linestyles='dotted')
    
    ax.fill_between(rango, 0, 30/rango,
                 facecolor="blue", # The fill color
                 color=color_verde,       # The outline color
                 alpha=0.02) 
    ax.fill_between(rango, 30/rango, 100/rango,
                 facecolor='blue', # The fill color
                 color=color_amarillo,       # The outline color
                 alpha=0.02) 
    ax.fill_between(rango,100/rango,10,
                 facecolor="blue", # The fill color
                 color=color_rojo,       # The outline color
                 alpha=0.02)
    
    
    ax.set_title(f' Evolución de Empiric Potential Growth en Badajoz\n Fuente del modelo @biocomsc \n Fuente de los datos #escovid19data', fontsize=12)
    

    
lista_elementos = []    
fig = plt.Figure(figsize=(10, 5), dpi=144)
ax = fig.add_subplot()


anim = FuncAnimation(fig=fig, func=update, init_func=init, frames=len(datos), 
                     interval=150, repeat=False)

#con interval puedes modular la velocidad del video, más alto --> mas lento

In [26]:
from IPython.display import HTML
html = anim.to_html5_video()
HTML(html)

<ipython-input-25-611948b128eb>:90: RuntimeWarning: divide by zero encountered in true_divide
  ax.fill_between(rango, 0, 30/rango,
<ipython-input-25-611948b128eb>:94: RuntimeWarning: divide by zero encountered in true_divide
  ax.fill_between(rango, 30/rango, 100/rango,
<ipython-input-25-611948b128eb>:98: RuntimeWarning: divide by zero encountered in true_divide
  ax.fill_between(rango,100/rango,10,


In [27]:
anim.save('modelo_biocomsc_badajoz.mp4',dpi=288)

<ipython-input-25-611948b128eb>:90: RuntimeWarning: divide by zero encountered in true_divide
  ax.fill_between(rango, 0, 30/rango,
<ipython-input-25-611948b128eb>:94: RuntimeWarning: divide by zero encountered in true_divide
  ax.fill_between(rango, 30/rango, 100/rango,
<ipython-input-25-611948b128eb>:98: RuntimeWarning: divide by zero encountered in true_divide
  ax.fill_between(rango,100/rango,10,
